In [ ]:
%pip install -U langchain langgraph langchain-community chromadb tiktoken gradio langchain_openai langchain-openai


In [ ]:
# =========================================================
# INSTALL (run once if needed)
# =========================================================
# !pip install -U langgraph langchain langchain-openai langchain-community chromadb gradio tiktoken

# =========================================================
# IMPORTS
# =========================================================
from typing import List, TypedDict
import tiktoken
import gradio as gr

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

from langchain_community.vectorstores import Chroma

# =========================================================
# STATE
# =========================================================
class ChatState(TypedDict):
    messages: List[BaseMessage]
    summary: str

# =========================================================
# LLM + TOKEN UTILS
# =========================================================
llm = ChatOpenAI(model="gpt-4o", temperature=0)

encoder = tiktoken.encoding_for_model("gpt-4o")

def token_count(messages: List[BaseMessage]) -> int:
    return len(encoder.encode(" ".join(m.content for m in messages)))

MAX_TOKENS = 2000
KEEP_LAST_N = 6

# =========================================================
# VECTOR MEMORY (FACTS)
# =========================================================
embeddings = OpenAIEmbeddings()

vectorstore = Chroma(
    collection_name="user_facts",
    embedding_function=embeddings,
    persist_directory="./chroma_db"
)

FACT_PROMPT = """
Extract explicit user facts (name, job, preferences).
Return one fact per line.
If none, return empty.
"""

def extract_and_store_facts(text: str):
    facts = llm.invoke(FACT_PROMPT + "\n\n" + text).content.strip()
    if not facts:
        return
    for fact in facts.split("\n"):
        vectorstore.add_texts([fact])

def retrieve_facts(query: str) -> str:
    docs = vectorstore.similarity_search(query, k=3)
    return "\n".join(d.page_content for d in docs)

# =========================================================
# SUMMARIZATION NODE
# =========================================================
SUMMARY_PROMPT = """
You are compressing conversation history.

Existing summary:
{summary}

New messages:
{content}

Preserve:
- user facts
- decisions
- goals
"""

def summarize_node(state: ChatState):
    messages = state["messages"]
    summary = state.get("summary", "")

    if token_count(messages) <= MAX_TOKENS:
        return state

    to_summarize = messages[:-KEEP_LAST_N]
    recent = messages[-KEEP_LAST_N:]

    content = "\n".join(m.content for m in to_summarize)

    new_summary = llm.invoke(
        SUMMARY_PROMPT.format(summary=summary, content=content)
    ).content

    return {
        "summary": new_summary,
        "messages": recent
    }

# =========================================================
# CHAT NODE
# =========================================================
def chat_node(state: ChatState):
    last_user_text = state["messages"][-1].content

    extract_and_store_facts(last_user_text)
    facts = retrieve_facts(last_user_text)

    system = SystemMessage(
        content=f"""
Conversation summary:
{state.get("summary", "")}

Long-term user facts:
{facts}
"""
    )

    response = llm.invoke([system] + state["messages"])

    return {
        "messages": state["messages"] + [
            AIMessage(content=response.content)
        ]
    }

# =========================================================
# GRAPH
# =========================================================
def should_summarize(state: ChatState):
    return "summarize" if token_count(state["messages"]) > MAX_TOKENS else END

graph = StateGraph(ChatState)
graph.add_node("chat", chat_node)
graph.add_node("summarize", summarize_node)

graph.set_entry_point("chat")

graph.add_conditional_edges(
    "chat",
    should_summarize,
    {
        "summarize": "summarize",
        END: END
    }
)

graph.add_edge("summarize", END)

# =========================================================
# CHECKPOINTING (IN-MEMORY)
# =========================================================
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

# =========================================================
# GRADIO UI
# =========================================================
config = {"configurable": {"thread_id": "user-1"}}

def chat(user_input, history):
    print("Checkpoint state before", checkpointer.get(config))
    result = app.invoke(
        {"messages": [HumanMessage(content=user_input)]},
        config=config
    )
    print("Checkpoint state after", checkpointer.get(config))
    print(result)
    return result["messages"][-1].content

print("Checking state", checkpointer)
gr.ChatInterface(
    chat,
    title="LangGraph Chatbot (Summary + Vector Memory + Checkpointing)"
).launch()


Checking state <langgraph.checkpoint.memory.InMemorySaver object at 0x147a546a0>
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Checkpoint state before None
Checkpoint state after {'v': 4, 'ts': '2026-01-16T05:25:45.488119+00:00', 'id': '1f0f29bc-efac-6e46-8001-7d19a46bab28', 'channel_versions': {'__start__': '00000000000000000000000000000002.0.4694003134385025', 'messages': '00000000000000000000000000000003.0.42328498149452565', 'branch:to:chat': '00000000000000000000000000000003.0.42328498149452565'}, 'versions_seen': {'__input__': {}, '__start__': {'__start__': '00000000000000000000000000000001.0.9703901308038636'}, 'chat': {'branch:to:chat': '00000000000000000000000000000002.0.4694003134385025'}}, 'updated_channels': ['messages'], 'channel_values': {'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]}}
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today